In [ ]:
import pandas as pd
import numpy as np
from scipy import stats


TrainDataFrame = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/colab datas/Boston.csv')
TestDataFrame = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/colab datas/bostontest.csv')

Predictors =['rm', 'ptratio', 'lstat', 'rad', 'zn']
#Predictors = ['rm', 'ptratio', 'lstat', 'rad', 'zn', 'tax', 'dis', 'nox']
TargetVariable='medv'


TrainDataForML = TrainDataFrame[Predictors].values
X = TrainDataForML

TestDataForML = TestDataFrame[Predictors].values

TrainPredDataForML = TrainDataFrame[[TargetVariable]].values
y = TrainPredDataForML
TestPredDataForML = TestDataFrame[[TargetVariable]].values


X_train=TrainDataForML
y_train= TrainPredDataForML

X_test=TestDataForML
y_test=TestPredDataForML

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(506, 5)
(506, 1)
(149, 5)
(149, 1)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
### Standardization of Data ###
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#Choose either Standardisation or Normalisation
# On this data Min and Max Normalisation produced better results

#Choose between standardisation and Min Max normalisation
#PredictorScaler = StandardScalar()
PredictorScalar = MinMaxScaler()

#Storing the fit object for later reference
PredictorScalarFit = PredictorScalar.fit(X)
PredictorScalarFitTest = PredictorScalar.fit(X_test)

#print(PredictorScalarFit)
print(PredictorScalarFitTest) 

#Generating the Standardised values of X
X = PredictorScalarFit.transform(X)
X_test = PredictorScalarFit.transform(X_test)
print(X)


MinMaxScaler(copy=True, feature_range=(0, 1))
[[0.52811118 0.09230769 0.09250694 0.         0.18      ]
 [0.47946936 0.47692308 0.22078323 0.14285714 0.        ]
 [0.72078332 0.47692308 0.06321307 0.14285714 0.        ]
 ...
 [0.65476943 0.96923077 0.11285846 0.         0.        ]
 [0.59728364 0.96923077 0.13876041 0.         0.        ]
 [0.35596968 0.96923077 0.18193031 0.         0.        ]]


In [ ]:
# Multiple Linear Regression
from sklearn.linear_model import LinearRegression
RegModel = LinearRegression()

# Printing all the parameters of Linear regression
print(RegModel)

# Creating the model on Training Data
LREG=RegModel.fit(X_train,y_train)
prediction=LREG.predict(X_test)

from sklearn import metrics
#Measuring Goodness of fit in Training Data
print('R2 Value:', metrics.r2_score(y_train,LREG.predict(X_train)))

#######################################################################

print('\n##### Model Validation and Accuracy Calculations ########')

#printing the sample values for prediction
TestingDataResults = pd.DataFrame(data=X_test, columns=Predictors) 
print(TestingDataResults)
TestingDataResults[TargetVariable]=y_test
TestingDataResults[('Predicted'+TargetVariable)]=np.round(prediction)

#Printing sample prediction values
print(TestingDataResults[[TargetVariable,'Predicted'+TargetVariable]].head())

#calculating the error for each row
#Mean Absolute Percentage Error

TestingDataResults['APE'] = 100 * ((abs(TestingDataResults['medv']-TestingDataResults['Predictedmedv']))/TestingDataResults['medv'])


MAPE = np.mean(TestingDataResults['APE'])
MedianMAPE = np.median(TestingDataResults['APE'])

Accuracy = 100 - MAPE
MedianAccuracy=100 - MedianMAPE

print('Mean Accuracy of Test Data:',Accuracy) # can be negative sometimes due to outlier
print('Median Accuracy of Test Data:',MedianAccuracy)

#Defining custom function to calculate Accuracy
#Make sure that there are no Zeros in the Target variable if you are using MAPE
def Accuracy_score(orig,pred):
  MAPE = np.mean(100*(np.abs(orig-pred)/orig))
  #print('#'*70,'Accuracy:',100-MAPE)
  return(100-MAPE)

#Custom Scoring MAPE Calculation
from sklearn.metrics import make_scorer
custom_Scoring = make_scorer(Accuracy_score, greater_is_better=True)

#Importing cross validation function frok sklearn
from sklearn.model_selection import  cross_val_score

#Running 10-fold cross validation on a given algorithm  
#Passing the complete data X and y because K-Fold will split the data automatically choose train/test 
Accuracy_Values = cross_val_score(RegModel,X,y,cv=10,scoring=custom_Scoring)
print('\n Accuracy values for 10-Fold Cross Validation: \n',Accuracy_Values)
print('\n Final Average Accuracy of the model: \n',round(Accuracy_Values.mean(),2))


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
R2 Value: 0.6790940548169776

##### Model Validation and Accuracy Calculations ########
           rm   ptratio     lstat       rad    zn
0    0.528111  0.092308  0.092507  0.000000  0.18
1    0.479469  0.476923  0.220783  0.142857  0.00
2    0.720783  0.476923  0.063213  0.142857  0.00
3    0.661718  0.615385  0.029602  0.285714  0.00
4    0.708781  0.615385  0.103299  0.285714  0.00
..        ...       ...       ...       ...   ...
144  0.000000  0.000000  0.842121  0.571429  0.00
145  0.387555  0.000000  0.796176  0.571429  0.00
146  0.228996  0.000000  0.452359  0.571429  0.00
147  0.007265  0.000000  0.849522  0.571429  0.00
148  0.089387  0.000000  0.812211  0.571429  0.00

[149 rows x 5 columns]
   medv  Predictedmedv
0  24.0           22.0
1  21.6           21.0
2  34.7           22.0
3  33.4           22.0
4  36.2           22.0
Mean Accuracy of Test Data: 83.38863550582245
Median Accuracy of Test 

In [ ]:
#POLYNOMIAL REGRESSION
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=1)
X_poly = poly_features.fit_transform(X_train)
LRpoly = LinearRegression()
LRpoly1=LRpoly.fit(X_poly,y_train)
Y_trainpoly_pred=LRpoly1.predict(X_poly)
y_test_predict = LRpoly1.predict(poly_features.fit_transform(X_test))
#print('Ytestpredict', y_test_predict)
#rmse=np.sqrt(y_train,Y_poly_pred)
r2 = r2_score(y_train,Y_trainpoly_pred)
#print("poly rmse =", rmse)
print("poly R2=",r2)

#plt.scatter(X, y, s=10)

print('\n##### Model Validation and Accuracy Calculations ##########')

# Printing some sample values of prediction
TestingDataResults=pd.DataFrame(data=X_test, columns=Predictors)
TestingDataResults[TargetVariable]=y_test
TestingDataResults[('PredictedPoly'+TargetVariable)]=np.round(y_test_predict)

# Printing sample prediction values
print(TestingDataResults[[TargetVariable,'PredictedPoly'+TargetVariable]].head())

# Calculating the error for each row
# mean absolute percentage error MAPE
TestingDataResults['APE']=100 * ((abs(
TestingDataResults['medv']-TestingDataResults['PredictedPolymedv']))/TestingDataResults['medv'])

MAPE=np.mean(TestingDataResults['APE'])
MedianMAPE=np.median(TestingDataResults['APE'])

Accuracy =100 - MAPE
MedianAccuracy=100- MedianMAPE
print('Mean Accuracy on test data:', Accuracy) # Can be negative sometimes due to outlier
print('Median Accuracy on test data:', MedianAccuracy)

def Accuracy_Score(orig,pred):
    MAPE = np.mean(100 * (np.abs(orig-pred)/orig))
    return(100-MAPE)

# Custom Scoring MAPE calculation 
from sklearn.metrics import make_scorer
scoring=make_scorer(Accuracy_Score, greater_is_better=True)

# Importing cross validation function from sklearn
from sklearn.model_selection import cross_val_score
def create_polynomial_regression_model(degree):
 poly_features = PolynomialFeatures(degree=degree)
 X_poly = poly_features.fit_transform(X)
 poly = LinearRegression()
 return np.mean(cross_val_score(poly, X_poly, y, cv=15, scoring=scoring))
poly_cv = []
for i in range(1,5):
 poly_cv.append(create_polynomial_regression_model(i))
 cv=create_polynomial_regression_model(i)
 print("degree=", i, 'cv=',cv)


poly R2= 0.6790940548169776

##### Model Validation and Accuracy Calculations ##########
   medv  PredictedPolymedv
0  24.0               22.0
1  21.6               21.0
2  34.7               22.0
3  33.4               22.0
4  36.2               22.0
Mean Accuracy on test data: 83.38863550582245
Median Accuracy on test data: 87.86610878661088
degree= 1 cv= 79.33999292396464
degree= 2 cv= 84.52133642443265
degree= 3 cv= 82.52257676588015
degree= 4 cv= 66.97341638445813


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score


from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=4)
X_poly = poly_features.fit_transform(X_train)
LRpoly = LinearRegression()
LRpoly1=LRpoly.fit(X_poly,y_train)
Y_trainpoly_pred=LRpoly1.predict(X_poly)
y_test_predict = LRpoly1.predict(poly_features.fit_transform(X_test))

#rmse=np.sqrt(y_train,Y_poly_pred)
r2 = r2_score(y_train,Y_trainpoly_pred)
#print("poly rmse =", rmse)
print("poly R2=",r2)

#plt.scatter(X, y, s=10)

print('\n##### Model Validation and Accuracy Calculations ##########')

# Printing some sample values of prediction
TestingDataResults=pd.DataFrame(data=X_test, columns=Predictors)
TestingDataResults[TargetVariable]=y_test
TestingDataResults[('PredictedPoly'+TargetVariable)]=np.round(y_test_predict)

# Printing sample prediction values
print(TestingDataResults[[TargetVariable,'PredictedPoly'+TargetVariable]].head())

# Calculating the error for each row
# mean absolute percentage error MAPE
TestingDataResults['APE']=100 * ((abs(
TestingDataResults['medv']-TestingDataResults['PredictedPolymedv']))/TestingDataResults['medv'])

MAPE=np.mean(TestingDataResults['APE'])
MedianMAPE=np.median(TestingDataResults['APE'])

Accuracy =100 - MAPE
MedianAccuracy=100- MedianMAPE
print('Mean Accuracy on test data:', Accuracy) # Can be negative sometimes due to outlier
print('Median Accuracy on test data:', MedianAccuracy)

def Accuracy_Score(orig,pred):
    MAPE = np.mean(100 * (np.abs(orig-pred)/orig))
    return(100-MAPE)

# Custom Scoring MAPE calculation 
from sklearn.metrics import make_scorer
scoring=make_scorer(Accuracy_Score, greater_is_better=True)

# Importing cross validation function from sklearn
from sklearn.model_selection import cross_val_score
def create_polynomial_regression_model(degree):
 poly_features = PolynomialFeatures(degree=degree)
 X_poly = poly_features.fit_transform(X)
 poly = LinearRegression()
 return np.mean(cross_val_score(poly, X_poly, y, cv=15, scoring=scoring))
poly_cv = []
for i in range(1,4):
 poly_cv.append(create_polynomial_regression_model(i))
 cv=create_polynomial_regression_model(i)
 print("degree=", i, 'cv=',cv)



poly R2= 0.8979821704108273

##### Model Validation and Accuracy Calculations ##########
   medv  PredictedPolymedv
0  24.0              793.0
1  21.6              896.0
2  34.7              632.0
3  33.4              760.0
4  36.2              701.0
Mean Accuracy on test data: -5554.355057738135
Median Accuracy on test data: -5481.592039800995
degree= 1 cv= 79.33999292396464
degree= 2 cv= 84.52133642443265
degree= 3 cv= 82.52257676588015
